In [3]:
from linares_plot import *
import os
import datetime

In [40]:
df=pd.read_excel(os.getcwd() + '\\ANOTACIONES2.xlsx', sheet_name= 'Truco 1' )

In [41]:
df.head()

,tiempo,1a,1b,2a,2b,2c,3a,3b,4a,4b,...,34b,35a,35b,36a,36b,36c,37a,37b,38a,38b
0,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00:00:02,NaN,NaN,A7,NaN,NaN,a4,NaN,a4,NaN,...,NaN,NaN,NaN,A5,NaN,NaN,NaN,NaN,A5,NaN
2,00:00:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,a4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00:00:06,Se,NaN,NaN,NaN,NaN,O,NaN,NaN,NaN,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,00:00:08,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN


In [43]:
df[['tiempo', '1a']]

,tiempo,1a
0,00:00:00,NaN
1,00:00:02,NaN
2,00:00:04,NaN
3,00:00:06,Se
4,00:00:08,NaN
5,00:00:10,Si
6,00:00:12,N
7,00:00:14,NaN
8,00:00:16,NaN
9,00:00:18,NaN


In [37]:
l = df.tiempo.iloc[158### hacer columna de tiempo en segundos
times_s=[]
for i in range(0, len(df)):
    l = df.tiempo.iloc[158]
    times_s.append(  int(datetime.timedelta(hours=l.hour, minutes=l.minute, seconds=l.second).total_seconds()) )



df['time_s'] = times_s
df.head()]
print(l.hour)
print(l.minute)
print(l.second)

0
5
16


In [38]:
l = df.tiempo.iloc[158]
int(datetime.timedelta(hours=l.hour, minutes=l.minute, seconds=l.second).total_seconds())

316

In [11]:
from datetime import datetime

datetime.strptime(df.tiempo.iloc[0], "%H:%M:%S")

TypeError: strptime() argument 1 must be str, not datetime.time